In [58]:
import numpy as np
import sys
import os
from math import ceil
%matplotlib qt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Conv3D, MaxPooling2D, Flatten, Input, Activation, BatchNormalization, Dropout, Reshape
import matplotlib.pyplot as plt
from tensorflow.keras.losses import categorical_crossentropy, MAPE
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [59]:
def create_cnn(width, height, depth, filters=(16, 16, 16, 16, 16)):
    input_shape = (height, width, depth)
    chan_dim = -1

    inputs = Input(shape=input_shape)
    for (i, filter) in enumerate(filters):
        if i == 0:
            x = inputs
            #x = Conv3D(filter, (3, 3, 1), padding="same", activation="relu")(x)
            #x = BatchNormalization(axis=chan_dim)(x)
            #x = Reshape(target_shape=(height,width,filter))(x)
        #else:
            # CONV => RELU => BN => POOL
        x = Conv2D(filter, (3, 3), padding="same", activation="relu")(x)
        x = BatchNormalization(axis=chan_dim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(16, activation="relu")(x)
    x = BatchNormalization(axis=chan_dim)(x)
    x = Dropout(0.5)(x)

    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(4, activation="linear")(x)

    # construct the CNN
    model = Model(inputs, x)

    # return the CNN
    return model

In [60]:
load_model = False
train_model = True
save_model = True
test_model = True

data_folder = 'thr_data'

In [61]:
print('creating model...')
model = create_cnn(256, 636, 1)
opt = Adam(lr=1e-3, decay=1e-3/200)
model.compile(loss="mean_absolute_error", optimizer=opt)
print(model.summary())

creating model...
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 636, 256, 1)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 636, 256, 16)      160       
_________________________________________________________________
batch_normalization_6 (Batch (None, 636, 256, 16)      64        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 318, 128, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 318, 128, 16)      2320      
_________________________________________________________________
batch_normalization_7 (Batch (None, 318, 128, 16)      64        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 159, 

In [62]:
if load_model:
    model.load_weights('thr_model.h5')

In [63]:
if train_model:
    print('loading training data...')
    vera_train = [f for f in os.listdir(data_folder) if f.endswith('-train-images.npy')]
    vico_train = [f for f in os.listdir(data_folder) if f.endswith('-train-angles.npy')]

    for im,an in zip(vera_train,vico_train):
        print(im)
        im_data = np.load('{0}/{1}'.format(data_folder,im))[...,None]
        an_data = np.load('{0}/{1}'.format(data_folder,an))[...,1:]
        print(im_data.shape,an_data.shape)

        print('training model...')
        model.fit(im_data, an_data, validation_split=0.1, epochs=10)

loading training data...
MidFlex-200205-01-train-images.npy
(978, 636, 256, 1) (978, 4)
training model...
Train on 880 samples, validate on 98 samples
Epoch 1/10
880/880 [==============================] - 82s 94ms/sample - loss: 0.9055 - val_loss: 0.7474
Epoch 2/10
880/880 [==============================] - 80s 91ms/sample - loss: 0.6844 - val_loss: 1.8163
Epoch 3/10
880/880 [==============================] - 80s 91ms/sample - loss: 0.5684 - val_loss: 2.9381
Epoch 4/10
880/880 [==============================] - 81s 92ms/sample - loss: 0.4783 - val_loss: 4.1214
Epoch 5/10
880/880 [==============================] - 81s 92ms/sample - loss: 0.4027 - val_loss: 3.7360
Epoch 6/10
880/880 [==============================] - 81s 92ms/sample - loss: 0.3580 - val_loss: 3.2475
Epoch 7/10
880/880 [==============================] - 81s 92ms/sample - loss: 0.3289 - val_loss: 2.5900
Epoch 8/10
880/880 [==============================] - 81s 92ms/sample - loss: 0.2934 - val_loss: 1.8119
Epoch 9/10
880/88

In [64]:
if save_model:
    print('saving model...')
    model.save('thr_model.h5', save_format='h5')

saving model...


In [65]:
if test_model:
    print('loading testing data...')
    vera_test = [f for f in os.listdir(data_folder) if f.endswith('-test-images.npy')]
    vico_test = [f for f in os.listdir(data_folder) if f.endswith('-test-angles.npy')]
    fingers = ['thumb','index','middle','ring','pinky']
    preds = np.zeros((420*len(vera_test),4))
    reals = np.zeros((420*len(vera_test),4))
    for i,(im,an) in enumerate(zip(vera_test,vico_test)):
        print(im)
        im_data = np.load('{0}/{1}'.format(data_folder,im))[...,None]
        an_data = np.load('{0}/{1}'.format(data_folder,an))[...,1:]
        print(im_data.shape,an_data.shape)

        print('testing model...')
        loss = model.evaluate(im_data, an_data, verbose=2)
        predictions = model.predict(im_data)
        preds[420*i:420*(i+1),...] = predictions
        reals[420*i:420*(i+1),...] = an_data
        for f,finger in enumerate(fingers[1:]):
            plt.subplot(len(vera_test),4,4*i+f+1)
            plt.plot(an_data[...,f]),plt.plot(predictions[...,f])
            if i==0:
                plt.title('{0} finger angle'.format(finger))
            if f==0:
                title = im.split('-')
                plt.ylabel('-'.join([title[i] for i in [0,2]]),rotation=90)
    plt.show()

loading testing data...
MidFlex-200205-01-test-images.npy
(420, 636, 256, 1) (420, 4)
testing model...
420/420 - 6s - loss: 0.3204
PinFlex-200205-01-test-images.npy
(420, 636, 256, 1) (420, 4)
testing model...
420/420 - 6s - loss: 0.4244
RinFlex-200205-01-test-images.npy
(420, 636, 256, 1) (420, 4)
testing model...
420/420 - 6s - loss: 0.3699
ThrExt-200205-01-test-images.npy
(420, 636, 256, 1) (420, 4)
testing model...
420/420 - 6s - loss: 0.2207


In [68]:
errs1 = np.sum(np.absolute(preds-reals),axis=0)/preds.shape[0]
errs2 = np.sum(np.absolute(preds-reals[...,0][...,None]),axis=0)/preds.shape[0]
errs3 = np.sum(np.absolute(preds-reals[...,1][...,None]),axis=0)/preds.shape[0]
errs4 = np.sum(np.absolute(preds-reals[...,2][...,None]),axis=0)/preds.shape[0]
errs5 = np.sum(np.absolute(preds-reals[...,3][...,None]),axis=0)/preds.shape[0]

print(errs1)

[0.46984805 0.24813758 0.32971035 0.28776812]


In [67]:
print(np.average(errs1))
print(np.average(errs2))
print(np.average(errs3))
print(np.average(errs4))
print(np.average(errs5))

0.33386602521834857
0.28604171060560024
0.3215410297271667
0.35632081047973047
0.36990101384832785
